In [1]:
from datasets import load_dataset, load_from_disk, concatenate_datasets
from promptsource import templates

CACHE_DIR = "/share/edc/home/antonis/datasets/huggingface"
import os
os.environ["HF_DATASETS_CACHE"] = CACHE_DIR

# # Get a list of all supported datasets
# datasets = templates.get_dataset_names()
# print(datasets)

/local/home/antonis/.conda/envs/incidental/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def print_example(example):
    for key in example.keys():
        print(f"{key}: {example[key]}")

In [ ]:
dataset = load_dataset('web_nlg', 'release_v3.0_en', cache_dir=CACHE_DIR)

In [ ]:
# ds_pth = "/share/edc/home/antonis/datasets/huggingface/merged_datasets/dataset_1/dataset_train.arrow"
# ds_pth = "/share/edc/home/antonis/datasets/huggingface/merged_datasets/sentiment/dataset_1/dataset_validation.arrow"
# ds_pth = "/share/edc/home/antonis/datasets/huggingface/merged_datasets/sentiment/P_QA_5/dataset_1/dataset_validation.arrow"
# dataset_c4 = load_dataset("c4", "en", split="train[:20%]", cache_dir=CACHE_DIR)

ds_pth = "/share/edc/home/antonis/datasets/huggingface/merged_datasets/sentiment_c4/P_1_PQA_5_promptsource_True/dataset_1/dataset_train.arrow"
dataset = load_from_disk(ds_pth)

ds_path2 = "/share/edc/home/antonis/datasets/huggingface/merged_datasets/sentiment_c4/P_1_PQA_5_promptsource_True/dataset_0/dataset_train.arrow"
dataset2 = load_from_disk(ds_path2)

In [ ]:
dataset[-5:]

In [ ]:
dataset2[-5:]

In [ ]:
ds_imdb = load_dataset("imdb", split="test", cache_dir=CACHE_DIR)
ds_sst = load_dataset("sst", split="test", cache_dir=CACHE_DIR)

print(f"IMDB len: {len(ds_imdb)}")
print(f"SST len: {len(ds_sst)}")

In [ ]:
# first protion is IMDB, second is SST
ds_pth_val = "/share/edc/home/antonis/datasets/huggingface/merged_datasets/sentiment_c4/P_1_PQA_5_promptsource_True/dataset_1/dataset_validation.arrow"
dataset_val = load_from_disk(ds_pth_val)
ds_pth2_val = "/share/edc/home/antonis/datasets/huggingface/merged_datasets/sentiment_c4/P_1_PQA_5_promptsource_True/dataset_0/dataset_validation.arrow"
dataset2_val = load_from_disk(ds_pth2_val)

In [ ]:
print(f"train1: {len(dataset)}")
print(f"train2: {len(dataset2)}")
print(f"val1: {len(dataset_val)}")
print(f"val2: {len(dataset2_val)}")

In [ ]:
dataset_val.select(range(len(dataset2_val)))

In [ ]:
# dataset_val[-len(dataset2_val):]

# new column

dataset_val1 = dataset_val.select(range(len(ds_imdb)))
dataset_val2 = dataset_val.select(range(len(ds_imdb), len(dataset_val)))

assert len(dataset_val1) + len(dataset_val2) == len(dataset_val), f"len mismatch: {len(dataset_val1)} + {len(dataset_val2)} != {len(dataset_val)}"

def add_new_column1(example, keyword, column_name="dataset"):
    example[column_name] = keyword
    return example

dataset_val1 = dataset_val1.map(lambda x: add_new_column1(x, "imdb"))
dataset_val2 = dataset_val2.map(lambda x: add_new_column1(x, "sst"))

dataset_val = concatenate_datasets([dataset_val1, dataset_val2])

# save the new dataset
dataset_val.save_to_disk(os.path.join(os.path.dirname(ds_pth_val), "dataset_validation_wname.arrow"))



In [ ]:
print(os.path.join(os.path.dirname(ds_pth_val), "dataset_validation_wname.arrow"))

In [6]:
import os
import json
import logging
from datasets import load_dataset, Dataset
from promptsource.templates import DatasetTemplates, TemplateCollection
# set logging level to INFO
from src._promptsource import get_T0MixtureDatasets 
logger = logging.getLogger(__name__)
logger.setLevel(20)

In [10]:
TOMixture = [
    ("sst","default"), # Senitment Classification")
    ("imdb",None),
]

In [2]:
# datasets = get_T0MixtureDatasets("test", max_samples=1000, return_as_dict=True)
# ds_sst = load_dataset("sst", "default", split="test", cache_dir=CACHE_DIR)
# ds_imdb = load_dataset("imdb", split="test", cache_dir=CACHE_DIR)
ds_yelp_review = load_dataset("yelp_review_full", split="train", cache_dir=CACHE_DIR)
ds_sentiment140 = load_dataset("sentiment140", split="train", cache_dir=CACHE_DIR)

Found cached dataset yelp_review_full (/share/edc/home/antonis/datasets/huggingface/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf)
Found cached dataset sentiment140 (/share/edc/home/antonis/datasets/huggingface/sentiment140/sentiment140/1.0.0/f81c014152931b776735658d8ae493b181927de002e706c4d5244ecb26376997)


In [14]:
name, subset = TOMixture[0]
example = ds_sst.select(range(1))
# DatasetTemplates(name, subset).templates.get_get_answer_choices_list(example)

In [24]:
print(len(ds_sst))
print(len(ds_imdb))
# print(len(ds_yelp_review))
# print(len(ds_sentiment140))

import random

ds_name = datasets['imdb']

def apply_template(example, template, dataset_name):
    if isinstance(template, list):
        template = random.choice(template)
    try:
        answer_choices = template.get_answer_choices_list(example)
    except:
        answer_choices = None
    # example['label'] = float(example['label'])
    return {"prompt": template.apply(example), "answer_choices": answer_choices, "dataset": dataset_name}

# # You can apply the template to the dataset
# prompted_dataset = ds_name.map(lambda example: apply_template(example, ds_name.templates))

def create_prompted_dataset(datasets):
    prompted_datasets = {}
    for name, dataset in datasets.items():
        prompted_dataset = dataset.map(lambda example: apply_template(example, dataset.templates, name))
        # columns to keep
        columns_to_keep = ["prompt", "answer_choices", "dataset"]
        prompted_dataset = prompted_dataset.remove_columns([col for col in prompted_dataset.column_names if col not in columns_to_keep])
        prompted_datasets[name] = prompted_dataset

    prompted_datasets = concatenate_datasets([dataset for name, dataset in prompted_datasets.items()])
    return prompted_datasets

prompted_datasets = create_prompted_dataset(datasets)

2210
25000


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [25]:
# save prompted dataset

# prompted_datasets.save_to_disk("/share/edc/home/antonis/datasets/huggingface/merged_datasets/sentiment_c4/P_1_PQA_5_promptsource_True/dataset_0/dataset_test_with_answers.arrow")

In [16]:
for name, dataset in datasets.items():
    print(name, dataset)
    print(dataset.templates)

sst/default Dataset({
    features: ['sentence', 'label', 'tokens', 'tree'],
    num_rows: 1000
})
[<promptsource.templates.Template object at 0x7f5180119590>, <promptsource.templates.Template object at 0x7f5128342550>, <promptsource.templates.Template object at 0x7f51285ab410>, <promptsource.templates.Template object at 0x7f51285ab150>, <promptsource.templates.Template object at 0x7f51285c0bd0>]
imdb Dataset({
    features: ['text', 'label'],
    num_rows: 1000
})
[<promptsource.templates.Template object at 0x7f51285867d0>, <promptsource.templates.Template object at 0x7f51285a6290>, <promptsource.templates.Template object at 0x7f51285de550>, <promptsource.templates.Template object at 0x7f51285d7750>, <promptsource.templates.Template object at 0x7f51283a3e90>, <promptsource.templates.Template object at 0x7f51285b4950>, <promptsource.templates.Template object at 0x7f51283b5b50>, <promptsource.templates.Template object at 0x7f51283b5ad0>, <promptsource.templates.Template object at 0x7f51